# 使用上下文变量创建一个基于经验的聊天

在这个例子中，我们展示了你如何通过发送和更新请求的上下文来建立一个简单的聊天机器人。

我们介绍了上下文变量对象，在这个演示中，它的功能类似于一个键值存储，你可以在运行内核时使用。

上下文是本地的（即在你的计算机的RAM中），并且在这个Jupyter会话的寿命之外不会被持久化。

在未来的例子中，我们将展示如何在磁盘上持久化上下文，以便你可以将其带入你的应用程序。

在这个聊天场景中，当用户与机器人来回交谈时，上下文会被填充到对话的历史中。在内核的每一次新的运行中，上下文可以向人工智能提供其变量的内容。

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.14.547.1-preview"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.SemanticFunctions;
using Microsoft.SemanticKernel.Orchestration;

IKernel kernel = KernelBuilder.Create();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
if (useAzureOpenAI)
    kernel.Config.AddAzureTextCompletionService(model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAITextCompletionService(model, apiKey, orgId);

让我们定义一个提示，概述一个对话聊天机器人。

In [ ]:
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

var promptConfig = new PromptTemplateConfig
{
    Completion =
    {
        MaxTokens = 2000,
        Temperature = 0.7,
        TopP = 0.5,
    }
};

注册语义函数：

In [ ]:
var promptTemplate = new PromptTemplate(skPrompt, promptConfig, kernel);
var functionConfig = new SemanticFunctionConfig(promptConfig, promptTemplate);
var chatFunction = kernel.RegisterSemanticFunction("ChatBot", "Chat", functionConfig);

初始化上下文

In [ ]:
var context = kernel.CreateNewContext();

var history = "";
context["history"] = history;

机器人聊天

In [ ]:
var userInput = "Hi, I'm looking for book suggestions";
context["userInput"] = userInput;

var bot_answer = await chatFunction.InvokeAsync(context);

用输出内容来更新 history

In [ ]:
history += $"\nUser: {userInput}\nMelody: {bot_answer}\n";
context.Variables.Update(history);

Console.WriteLine(context);

继续聊天吧！

In [ ]:
Func<string, Task> Chat = async (string input) => {
    // Save new message in the context variables
    context["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(context);

    // Append the new interaction to the chat history
    history += $"\nUser: {input}\nMelody: {answer}\n"; 
    context["history"] = history;
    
    // Show the response
    Console.WriteLine(context);
};

await Chat("I would like a non-fiction book suggestion about Greece history. Please only list one book.");
await Chat("that sounds interesting, what are some of the topics I will learn about?");
await Chat("Which topic from the ones you listed do you think most people find interesting?");
await Chat("could you list some more books I could read about the topic(s) you mentioned?");